# MNIST Odd-Even classification with TensorFlow (CNN)

## Data loading

Let's load the dataset using the Keras API implementation included in TensorFlow.

As we only want to classify the images between **odd** and **even** numbers, we will map labels to *1* if odd or *0* if even.

In [ ]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Keep original test numbers for displaying
y_numbers = y_test

# Set Even or Odd labels for each sample
y_train = np.array(list(map(lambda x: [1, 0] if x%2 else [0, 1], y_train)), dtype=np.float32)
y_test = np.array(list(map(lambda x: [1, 0] if x%2 else [0, 1], y_test)), dtype=np.float32)

# Normalize images dividing by max pixel value (255)
x_train = (x_train / 255.0).astype(np.float32)
x_test = (x_test / 255.0).astype(np.float32)

# Reshape to TF API (#img, rows, cols, channels)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

## Hyperparameters

In [ ]:
EPOCHS = 5
BATCH_SIZE = 120
LEARNING_RATE = 0.01

## CNN Model

### Placeholders

In [ ]:
# Placeholder for input data formatted as N*784 tensors (image_size = 28*28 = 784)
x = tf.placeholder(name='x', dtype=tf.float32, shape=(None, 784))
# Input data in image format for Convolutional Layers
x_image = tf.reshape(x, shape=(-1, 28, 28, 1))

# Placeholder for labels formatted as N*2 tensors (2 classes -> odd | even)
y_true = tf.placeholder(name="y_true", dtype=tf.float32, shape=(None, 2))
# To calculate true class
y_true_class = tf.argmax(y_true, axis=1)

In [ ]:
# Load data into a Dataset placeholder
features_placeholder = tf.placeholder(x_train.dtype, x_train.shape)
labels_placeholder = tf.placeholder(y_train.dtype, y_train.shape)

# Create TF.Dataset from the data, making it repeatable, grouping by batch size and shuffling it in each epoch
training_set = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
training_set = training_set.batch(BATCH_SIZE)
training_set = training_set.repeat(EPOCHS)
training_set = training_set.shuffle(len(x_train))

# Iterator to deal with the dataset in the training loop
training_set_iterator = training_set.make_initializable_iterator()

### Neural Network architecture

We will use the classical architecture for a CNN (Convolution + Pooling > Convolution + Pooling > Flatten > Dense > Dense)

Instead of the common Convolution > ReLU > Pooling, we can apply the ReLU after Pooling step beacuse ReLU(MaxPooling(x)) == MaxPooling(ReLU(x)) and it will be more efficient, as the tensor size will be 75% smaller.

In [ ]:
# 1st Convolutional Layer 32x5x5 Kernel
conv1 = tf.layers.conv2d(x_image,
                         filters=32,
                         kernel_size=(5, 5),
                         padding='same')

# Apply MaxPooling 2x2
pool1 = tf.layers.max_pooling2d(conv1,
                                pool_size=(2, 2),
                                strides=2)

# Apply ReLU activation function
relu1 = tf.nn.relu(pool1)

In [ ]:
# 2nd Convolutional Layer 64x5x5 Kernel
conv2 = tf.layers.conv2d(relu1,
                         filters=64,
                         kernel_size=(5, 5),
                         padding='same')

# Apply MaxPooling 2x2
pool2 = tf.layers.max_pooling2d(conv2,
                                pool_size=(2, 2),
                                strides=2)

# Apply ReLu activation function
relu2 = tf.nn.relu(pool2)

In [ ]:
# Apply Dropout p=0.25
dropout1 = tf.layers.dropout(relu2, rate=0.25)

# Flatten data into (-1, 7*7*64 tensor)
flatten = tf.reshape(dropout1, shape=(-1, 7*7*64))

In [ ]:
# Dense Layer 1 with 100 neurons
dense1 = tf.layers.dense(flatten, units=100, activation=tf.nn.relu)

# Apply Dropout p=0.5
dropout2 = tf.layers.dropout(dense1, rate=0.5)

In [ ]:
# Dense Layer 2 with 2 neurons (our final output)
dense2 = tf.layers.dense(dense1, units=2)

In [ ]:
# Prediction
y_pred = tf.nn.softmax(dense2)

# Predicted class
y_pred_class = tf.argmax(y_pred, axis=1)

### Loss function

In [ ]:
# Cross-Entropy as loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=dense2, labels=y_true)
cost = tf.reduce_mean(cross_entropy)

### Optimizer

In [ ]:
# Use ADAM optimizer to minimize the loss function defined previously
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)

### Performance evaluation

In [ ]:
correct_prediction = tf.equal(y_pred_class, y_true_class)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

### Train loop

In [ ]:
def train(session):
    next_element = training_set_iterator.get_next()
    
    num_iterations = EPOCHS * (len(x_train) // BATCH_SIZE)
    for it in range(1, num_iterations + 1):
        x_batch, y_true_batch = session.run(next_element)

        feed_dict_train = { x_image: x_batch,
                            y_true: y_true_batch }
        session.run(optimizer, feed_dict=feed_dict_train)
        
        if it % 100 == 0:
            acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = 'Iteration: {:>4}, Training Accuracy: {:>6.2%}'
            print(msg.format(it, acc))

### Evaluate performance

In [ ]:
def test(session):
    predictions = []
    correct = 0
    for image, label in zip(x_test, y_test):
        feed_dict = { x_image: image.reshape(-1, 28, 28, 1),
                      y_true: label.reshape(-1, 2)}
                      
        prediction_class = session.run(y_pred_class, feed_dict=feed_dict)
        predictions.append(prediction_class[0])
        
        if (prediction_class[0] == np.argmax(label)):
            correct += 1
            
    acc = float(correct) / len(x_test)
    msg = '\nAccuracy on Test-Set: {0:.2%} ({1} / {2})'
    print(msg.format(acc, correct, len(x_test)))
    
    return predictions

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline

def plot_missclasifications(preds):
    count = 0
    plt.figure(figsize=(10, 10))
    for i, pred in enumerate(preds):
        if pred != np.argmax(y_test[i]):
            msg = '{} ({})'
            msg = msg.format('Even' if preds[i] else 'Odd', str(y_numbers[i]))
        
            plt.subplot(2, 5, count + 1)
            plt.title(msg)
            plt.axis('off')
            plt.imshow(x_test[i].reshape(28, 28), cmap=cm.binary)
            count += 1
            if count == 10:
                break

### Run CNN in a session

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(training_set_iterator.initializer, feed_dict={ features_placeholder: x_train,
                                                            labels_placeholder: y_train })
    
    train(sess)
    predictions = test(sess)

Some missclassification examples

In [ ]:
plot_missclasifications(predictions)